# Prepare timeseries data FOR LEVICO

-----
**TASK**
- control the consistency of the time series
- control the quality of data
- resample the time series




## Work directory

`Precipitation`

---`Raw_data`: data downloaded from meteo services

---`Checked_data`: checked data (time series consistency, quality) 

---`Hourly_data`: checked data resample at hourly time scale   

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
#import datetime as datetime
from datetime import datetime, timedelta

from plotly import graph_objs as go
import calendar
from plotly.offline import plot, iplot
import plotly.graph_objs as go
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

work_directory_path = os.getcwd()
print('\t', work_directory_path)

	 C:\Users\super\OMS_Project_GEOET11\OMS_Project_GEOET11\JupyterNotebook


In [2]:
oms_project_path = os.path.dirname(os.getcwd())
os.chdir(oms_project_path+'\\data\\Levico')

In [3]:
os.listdir()

['1',
 '2',
 'Cost9999.csv',
 'DATASS_Levico_1H.csv',
 'Net_1.csv',
 'Net_allsky_1.csv',
 'Pressione_Levico.csv',
 'Pressione_Levico_0.csv',
 'Prova_sole.csv',
 'RadSolTot_Levico.csv',
 'Sin_.csv',
 'Temp_Levico.csv',
 'Temp_Levico_0.csv',
 'UmiAriPerc_Levico.csv',
 'UmiAriPerc_Levico_0.csv',
 'VelVenMedia_Levico.csv',
 'VelVenMedia_Levico_0.csv']

## Read the file 



In [4]:
filename='Pressione_Levico'

In [5]:
raw_data = pd.read_csv(filename+'.csv', skiprows = 4, names=['Datetime','Value','Quality','Metadata'], low_memory=False,encoding= 'unicode_escape')
raw_data['Datetime'] = pd.to_datetime(raw_data['Datetime'],dayfirst=True)
raw_data.head()

,Datetime,Value,Quality,Metadata
0,2004-08-17 11:30:00,955.9,1.0,Sites:
1,2004-08-17 11:45:00,956.1,1.0,T0010 - Levico (Terme) Lat:46.010558 Long:11.3...
2,2004-08-17 12:00:00,956.1,1.0,NaN
3,2004-08-17 12:15:00,956.1,1.0,Variables:
4,2004-08-17 12:30:00,955.9,1.0,550 - Pressione atmosferica (Ettopascal)


In [6]:
raw_data.tail()

,Datetime,Value,Quality,Metadata
893933,2022-05-31 06:50:00,0.6,145.0,NaN
893934,2022-05-31 07:00:00,0.4,145.0,NaN
893935,2022-05-31 07:10:00,0.4,145.0,NaN
893936,2022-05-31 07:20:00,0.4,145.0,NaN
893937,2022-05-31 07:30:00,0.4,145.0,NaN


## Date format

In [7]:
print('INDEX DATE')
print(0, ' ', pd.to_datetime(raw_data['Datetime'][0]))
print(1, ' ', pd.to_datetime(raw_data['Datetime'][1]))
print(2, ' ', pd.to_datetime(raw_data['Datetime'][2]))
print(3, ' ', pd.to_datetime(raw_data['Datetime'][3]))

INDEX DATE
0   2004-08-17 11:30:00
1   2004-08-17 11:45:00
2   2004-08-17 12:00:00
3   2004-08-17 12:15:00


**PROBLEM:** index `2` is not correct. It should be 2004-01-01.

**SOLUTION**: specify date format

In [8]:
print('INDEX DATE')
print(0, ' ', pd.to_datetime(raw_data['Datetime'][0],format='%H:%M:%S %d/%m/%Y'))
print(1, ' ', pd.to_datetime(raw_data['Datetime'][1],format='%H:%M:%S %d/%m/%Y'))
print(2, ' ', pd.to_datetime(raw_data['Datetime'][2],format='%H:%M:%S %d/%m/%Y'))
print(3, ' ', pd.to_datetime(raw_data['Datetime'][3],dayfirst=True))

INDEX DATE
0   2004-08-17 11:30:00
1   2004-08-17 11:45:00
2   2004-08-17 12:00:00
3   2004-08-17 12:15:00


or use the argument `dayfirst`

In [9]:
print('INDEX DATE')
print(0, ' ', pd.to_datetime(raw_data['Datetime'][0],dayfirst=True))
print(1, ' ', pd.to_datetime(raw_data['Datetime'][1],dayfirst=True))
print(2, ' ', pd.to_datetime(raw_data['Datetime'][2],dayfirst=True))
print(3, ' ', pd.to_datetime(raw_data['Datetime'][3],dayfirst=True))

INDEX DATE
0   2004-08-17 11:30:00
1   2004-08-17 11:45:00
2   2004-08-17 12:00:00
3   2004-08-17 12:15:00


**COMPLETE SCRIPT TO READ A TIMESERIES**

In [10]:
#raw_data['Datetime'] = pd.to_datetime(raw_data['Datetime'],dayfirst=True)
#raw_data.head()

In [11]:
raw_data.tail()

,Datetime,Value,Quality,Metadata
633313,2022-05-31 06:30:00,955.8,145.0,NaN
633314,2022-05-31 06:45:00,955.9,145.0,NaN
633315,2022-05-31 07:00:00,956.1,145.0,NaN
633316,2022-05-31 07:15:00,956.1,145.0,NaN
633317,2022-05-31 07:30:00,956.3,145.0,NaN


## Check if there are dups in the timeseries

In [12]:
boolean = raw_data.duplicated(subset=['Datetime']).any()
boolean

True

In [13]:
## DEAL WITH DUPS
#raw_data = raw_data[~raw_data.index.duplicated(keep='first')] # keep the first value
raw_data = raw_data[~raw_data.index.duplicated(keep='last')] # keep the last value

## Data quality check

The meteo file contains also an information about the quality of the data:
- 1 dato buono
- 90 Neve (ritardato)
- 140 dato incerto
- 145 da teletrasmissione, non validato
- 151 dato mancante
- 255 No data

This information must be used to prepare the input data.

In [14]:
quality_check = raw_data[['Datetime','Value','Quality']].copy()
#quality_check.head()

In [15]:
for i in range(1,quality_check.shape[0]):
    if quality_check['Quality'][i]>140:   #ATTENZIONE ERA !=1 pero posso mettere >145 per avere tutti i dati anche se sono incerti...
        quality_check['Value'][i] = np.nan

In [16]:
#quality_check.head()

In [17]:
#quality_check.tail()

## Timeseries consistency

Some errors may occur parsing dates

In [18]:
tmp = []
for i in range(0,quality_check.shape[0]-1):
    if (raw_data['Datetime'][i+1]<raw_data['Datetime'][i]):
        tmp.append(i)

if len(tmp)!=0:
    print('WARNING: the time series has some wrong dates')
    print(tmp)

It is possible that in your data the time step is not  multiple of 5 minutes

In [19]:
'''
tmp = []
for i in range(0,quality_check.shape[0]-1):
    if (raw_data['Datetime'][i+1]-raw_data['Datetime'][i])%5:
        tmp.append(i)

if len(tmp)!=0:
    print('WARNING: the time series has some wrong dates')
    '''

"\ntmp = []\nfor i in range(0,quality_check.shape[0]-1):\n    if (raw_data['Datetime'][i+1]-raw_data['Datetime'][i])%5:\n        tmp.append(i)\n\nif len(tmp)!=0:\n    print('WARNING: the time series has some wrong dates')\n    "

## Create a complete timeseries
A time series is defined by
- `start`: start date
- `end`: end date
- `freq`: frequency. Five minutes `'5min'`, 1 hour `'1H'`, 1 day `'1D'`

In [21]:
date_rng=pd.date_range(start='2000-01-01 11:30',end='2022-05-31 07:30',freq='15min')

In [22]:
date_rng

DatetimeIndex(['2000-01-01 11:30:00', '2000-01-01 11:45:00',
               '2000-01-01 12:00:00', '2000-01-01 12:15:00',
               '2000-01-01 12:30:00', '2000-01-01 12:45:00',
               '2000-01-01 13:00:00', '2000-01-01 13:15:00',
               '2000-01-01 13:30:00', '2000-01-01 13:45:00',
               ...
               '2022-05-31 05:15:00', '2022-05-31 05:30:00',
               '2022-05-31 05:45:00', '2022-05-31 06:00:00',
               '2022-05-31 06:15:00', '2022-05-31 06:30:00',
               '2022-05-31 06:45:00', '2022-05-31 07:00:00',
               '2022-05-31 07:15:00', '2022-05-31 07:30:00'],
              dtype='datetime64[ns]', length=785841, freq='15T')

In [23]:
# quality_check.reset_index(inplace=True)

In [24]:
# set the column time as the index of the dataframe
quality_check.set_index('Datetime', inplace=True)
quality_check.head()

,Value,Quality
Datetime,,
2004-08-17 11:30:00,955.9,1.0
2004-08-17 11:45:00,956.1,1.0
2004-08-17 12:00:00,956.1,1.0
2004-08-17 12:15:00,956.1,1.0
2004-08-17 12:30:00,955.9,1.0


## Join the dataframe with correct timeseries `df_dates` and the dataframe with corrected data `quality_check` with the function `pd.join()`

In [25]:
df_dates = pd.DataFrame({'Time': date_rng})
df_dates.set_index('Time',inplace=True)


In [26]:
df_dates = df_dates.join(quality_check, how='outer')
df_dates.head(15)

,Value,Quality
2000-01-01 11:30:00,NaN,NaN
2000-01-01 11:45:00,NaN,NaN
2000-01-01 12:00:00,NaN,NaN
2000-01-01 12:15:00,NaN,NaN
2000-01-01 12:30:00,NaN,NaN
2000-01-01 12:45:00,NaN,NaN
2000-01-01 13:00:00,NaN,NaN
2000-01-01 13:15:00,NaN,NaN
2000-01-01 13:30:00,NaN,NaN
2000-01-01 13:45:00,NaN,NaN


In [27]:
df_dates.tail()

,Value,Quality
2022-05-31 06:30:00,NaN,145.0
2022-05-31 06:45:00,NaN,145.0
2022-05-31 07:00:00,NaN,145.0
2022-05-31 07:15:00,NaN,145.0
2022-05-31 07:30:00,NaN,145.0


Control the cumulate of raw data and corrected one

In [28]:
if raw_data['Value'].sum()-df_dates['Value'][df_dates['Value']>0.0].sum()<10e-10:
    print('OK')

In [29]:
#df_dates.loc['2004-8-18 13:00']

In [30]:
df_dates.index

DatetimeIndex(['2000-01-01 11:30:00', '2000-01-01 11:45:00',
               '2000-01-01 12:00:00', '2000-01-01 12:15:00',
               '2000-01-01 12:30:00', '2000-01-01 12:45:00',
               '2000-01-01 13:00:00', '2000-01-01 13:15:00',
               '2000-01-01 13:30:00', '2000-01-01 13:45:00',
               ...
               '2022-05-31 05:15:00', '2022-05-31 05:30:00',
               '2022-05-31 05:45:00', '2022-05-31 06:00:00',
               '2022-05-31 06:15:00', '2022-05-31 06:30:00',
               '2022-05-31 06:45:00', '2022-05-31 07:00:00',
               '2022-05-31 07:15:00', '2022-05-31 07:30:00'],
              dtype='datetime64[ns]', length=858129, freq=None)

## *Si può vedere il grafico dei dati*

In [31]:
'''
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_dates.index, y=df_dates['Value'], name="[unit]",
                         line_color='deepskyblue'))
# fig.add_trace(go.Scatter(x=df.Date, y=df['AAPL.Low'], name="AAPL Low",
#                          line_color='dimgray'))

fig.update_layout(title_text='Station Levico',
                  xaxis_rangeslider_visible=True)
fig.show()
'''

'\nfig = go.Figure()\nfig.add_trace(go.Scatter(x=df_dates.index, y=df_dates[\'Value\'], name="[unit]",\n                         line_color=\'deepskyblue\'))\n# fig.add_trace(go.Scatter(x=df.Date, y=df[\'AAPL.Low\'], name="AAPL Low",\n#                          line_color=\'dimgray\'))\n\nfig.update_layout(title_text=\'Station Levico\',\n                  xaxis_rangeslider_visible=True)\nfig.show()\n'

Change header `Value` with the code of the meteo station, and set NaN to -9999

In [32]:
#df_dates['Value']=df_dates['Value']/3.6 #CONVERSIONE kJ/m2 TO W/m2 SOLO PER RADIAZIONE SOLE

In [33]:
df_correct = df_dates.replace(np.nan,'-9999')
df_correct.rename(columns={"Value": "1"}, inplace=True)
df_correct.head()

,1,Quality
2000-01-01 11:30:00,-9999,-9999
2000-01-01 11:45:00,-9999,-9999
2000-01-01 12:00:00,-9999,-9999
2000-01-01 12:15:00,-9999,-9999
2000-01-01 12:30:00,-9999,-9999


In [34]:
df_correct.tail()

,1,Quality
2022-05-31 06:30:00,-9999,145.0
2022-05-31 06:45:00,-9999,145.0
2022-05-31 07:00:00,-9999,145.0
2022-05-31 07:15:00,-9999,145.0
2022-05-31 07:30:00,-9999,145.0


#### **FACCIO A MENO DI SALVARE QUESTO FILE INUTILE**

In [35]:
#df_correct.to_csv('C:\\Users\\super\\OMS_Project_GEOET11\\OMS_Project_GEOET11\\data\\Levico\\Temp_Levico_checked.csv', columns=['1'])

Google: _dataframe save csv specifying path_

https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas

In [36]:
#break

# Resample the timeseries !!

When resampling it is possible to use two different functions
- `sum`
- `mean`

For precipitation we cumulate hence we use `sum`, but for temperature and discharge? use *mean*

In [37]:
df_hourly = quality_check.resample('1H').mean() #sum(min_count=1)
df_hourly.head()

,Value,Quality
Datetime,,
2004-08-17 11:00:00,956.000000,1.0
2004-08-17 12:00:00,955.950000,1.0
2004-08-17 13:00:00,955.650000,1.0
2004-08-17 14:00:00,955.200000,1.0
2004-08-17 15:00:00,955.133333,1.0


## *Si crea la Serie Temporale con data iniziale e finale che voglio*

In [39]:
#Replace initial date and end date, considering also the frequency (H=hour)
date_rng=pd.date_range(start='2000-01-01 11:00',end='2022-05-31 07:30',freq='1H') 

In [40]:
df_dates = pd.DataFrame({'Time': date_rng})
df_dates.set_index('Time',inplace=True)

In [41]:
df_dates = df_dates.join(df_hourly, how='outer')
df_dates.head()

,Value,Quality
2000-01-01 11:00:00,NaN,NaN
2000-01-01 12:00:00,NaN,NaN
2000-01-01 13:00:00,NaN,NaN
2000-01-01 14:00:00,NaN,NaN
2000-01-01 15:00:00,NaN,NaN


## Conversione unità di Misura in Pascal **SOLO PER PRESSIONE**

In [42]:
df_dates['Value']=df_dates['Value']*100

### Conversione unità di Misura in Watts **SOLO PER RADIAZIONE!**

In [43]:
#df_dates['Value']=df_dates['Value']/3.6

## Sostituisce i valori *nan* con *-9999*

In [44]:
df_correct = df_dates.replace(np.nan,'-9999')
df_correct.rename(columns={"Value": "1"}, inplace=True)
df_correct.head()

,1,Quality
2000-01-01 11:00:00,-9999,-9999
2000-01-01 12:00:00,-9999,-9999
2000-01-01 13:00:00,-9999,-9999
2000-01-01 14:00:00,-9999,-9999
2000-01-01 15:00:00,-9999,-9999


In [45]:
df_correct.tail()

,1,Quality
2022-05-31 03:00:00,-9999,145.0
2022-05-31 04:00:00,-9999,145.0
2022-05-31 05:00:00,-9999,145.0
2022-05-31 06:00:00,-9999,145.0
2022-05-31 07:00:00,-9999,145.0


In [46]:
df_correct.to_csv(oms_project_path+'\\data\\Levico\\DATASS_Levico_1H.csv', columns=['1'])

TIPS:
- create a folder for precipitation, temperature, discharge
- for each variable, i.e. in each folder, create these subfolders: `Raw_data`, `Checked_data`, `Hourly_data`
    in `Raw_data` you collect all the files that you have downloaded, in `Checked_data` the checked time series (values, and complete dates), in `Hourly_data` the resampled time series
- rewrite this notebook as function, what do you have to modify to resample temperature and discharge data?
- use the command `for file in sorted(glob.glob('%s/*.csv' % inputFolder)):` to loop over the files within a folder


## Ho to write the time series for OMS

OMS time series has to be formatted as:

In [47]:
#Image(filename=work_directory_path+'/OMS_time_series.png',width=800, height=400)

Once you have prepared all your resampled data, create a unique dataframe for precipitation, temperature. This dataframe has to be formatted as: the first column you have your dates, in the following columns your station. The header of these columns must be the code of the station, the same that you use in the shapefile containing the meteo stations. Save this file as a .csv. This file can be processed with the function `gf_io.write_OMS_timeseries` to get the data formatted for OMS.

TIPS: write a simple script that loop over the file in `Precipitation/Hourly_data` and create a unique dataframe using the the method `pd.concat()` save this as `Hourly_data_no_formatted.csv`. The dataframe must have as index the dates as datatime.

More on `merge`, `concat` and `join` at
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [48]:
'''
df1 = pd.read_csv('C:\\Users\\super\\OMS_Project_GEOET11\\OMS_Project_GEOET11\\data\\Levico\\Temp_Levico_1H.csv', low_memory=False)
df1.rename(columns={ df1.columns[0]: "Datetime" }, inplace = True)
df1['Datetime'] = pd.to_datetime(df1['Datetime'],dayfirst=True)
df1.set_index('Datetime', inplace=True)
# df1.head()
'''

'\ndf1 = pd.read_csv(\'C:\\Users\\super\\OMS_Project_GEOET11\\OMS_Project_GEOET11\\data\\Levico\\Temp_Levico_1H.csv\', low_memory=False)\ndf1.rename(columns={ df1.columns[0]: "Datetime" }, inplace = True)\ndf1[\'Datetime\'] = pd.to_datetime(df1[\'Datetime\'],dayfirst=True)\ndf1.set_index(\'Datetime\', inplace=True)\n# df1.head()\n'

### **CAMBIA NOME AL FILE!**

In [49]:
df2 = pd.read_csv(oms_project_path+'\\data\\Levico\\DATASS_Levico_1H.csv', low_memory=False)
df2.rename(columns={ df2.columns[0]: "Datetime" }, inplace = True)
df2.rename(columns={ df2.columns[1]: "1" }, inplace = True) # just to have a different header, this is not necessary when you will work on your data
df2['Datetime'] = pd.to_datetime(df2['Datetime'],dayfirst=True)

df2.set_index('Datetime', inplace=True)

In [50]:
df = pd.concat([df2],axis=1)
df.to_csv(oms_project_path+'\\data\\Levico\\'+filename+'_0.csv')

In [51]:
df

,1
Datetime,
2000-01-01 11:00:00,-9999.0
2000-01-01 12:00:00,-9999.0
2000-01-01 13:00:00,-9999.0
2000-01-01 14:00:00,-9999.0
2000-01-01 15:00:00,-9999.0
...,...
2022-05-31 03:00:00,-9999.0
2022-05-31 04:00:00,-9999.0
2022-05-31 05:00:00,-9999.0


In [52]:
from geoframepy.timeseries import io_csv

In [53]:
df_noformatted = pd.read_csv(oms_project_path+'\\data\\Levico\\'+filename+'_0.csv', low_memory = False)
df_noformatted.head()

,Datetime,1
0,2000-01-01 11:00:00,-9999.0
1,2000-01-01 12:00:00,-9999.0
2,2000-01-01 13:00:00,-9999.0
3,2000-01-01 14:00:00,-9999.0
4,2000-01-01 15:00:00,-9999.0


In [54]:
df_noformatted.tail()

,Datetime,1
196456,2022-05-31 03:00:00,-9999.0
196457,2022-05-31 04:00:00,-9999.0
196458,2022-05-31 05:00:00,-9999.0
196459,2022-05-31 06:00:00,-9999.0
196460,2022-05-31 07:00:00,-9999.0


In [55]:
#help(io_csv.write_OMS_timeseries)

**NOTE:** `df_noformatted` must be passed without the first column. The first column is present in the database.csv to know which is the time series.

## **CAMBIA DATA INIZIO E NOME FILE!**

In [56]:
io_csv.write_OMS_timeseries(df_noformatted.iloc[:,1:], oms_project_path+'\\data\\Levico\\'+filename+'_0.csv', has_datetime=False, start_date='01-01-2000 11:00', frequency='1H')



***SUCCESS writing!  C:\Users\super\OMS_Project_GEOET11\OMS_Project_GEOET11\data\Levico\Pressione_Levico_0.csv
